In [33]:
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table

In [34]:
should_be_uploaded = pd.read_csv('should_be_uploaded.csv')

In [35]:
len(should_be_uploaded)

220801

In [36]:
latest_classifications = pd.read_csv('~/Downloads/classifications_final.csv')  # only if at least 1 classification

In [37]:

# latest_subjects = pd.read_csv('/home/walml/repos/gz-panoptes-reduction/data/latest_subjects_export.csv')
# latest_subjects = pd.read_csv('/media/walml/beta/misc_downloads/galaxy-zoo-subjects_2020_10_24.csv')
# latest_subjects = pd.read_csv('/media/walml/beta/misc_downloads/galaxy-zoo-subjects_2020_10_27.csv')
latest_subjects = pd.read_csv('/media/walml/beta/misc_downloads/galaxy-zoo-subjects_2020_10_28.csv')
latest_subjects = latest_subjects[latest_subjects['workflow_id'].isin([6122.0, 10582.0, 10581.0])]  # decals public workflows
# latest_subjects = latest_subjects[latest_subjects['workflow_id'].isin([10582.0])]  # decals enhanced
latest_subjects = latest_subjects[~latest_subjects['subject_set_id'].isin([77652])]  # remove eagle subjects
len(latest_subjects)

436559

In [38]:
latest_subjects['metadata'] = latest_subjects['metadata'].apply(json.loads)

def get_iauname(metadata):
    possible_keys = ['iauname', '!iauname', 'IAUNAME', '!IAUNAME']
    for key in possible_keys:
        if key in metadata.keys():
            return metadata[key]
    return np.nan

latest_subjects['iauname'] = latest_subjects['metadata'].apply(get_iauname)
assert pd.isna(latest_subjects['iauname']).sum()  == 0

In [39]:
latest_subjects['subject_set_id'].value_counts()

74905    357026
85299     21900
67383     15000
21156     10000
74807      5765
60259      5000
20603      5000
60995      5000
20036      3000
51756      3000
20156      2000
19832      1601
60258       977
20354       790
67382       500
Name: subject_set_id, dtype: int64

## Which subjects still require classifications?

- in should_be_uploaded
- < 5 classifications (iauname, including duplicates)



In [40]:
enough_classifications = latest_classifications[latest_classifications['smooth-or-featured_total-votes'] >= 5]['iauname']
needs_classifications = should_be_uploaded[~should_be_uploaded['iauname'].isin(enough_classifications)]
# this is ex

In [41]:
len(needs_classifications)

34971

We can subtract out the 9074 I just uploaded to missing_e, they will all have retirement=5 correctly

In [42]:
have_just_uploaded = pd.read_csv('missing_galaxies_no_classifications_not_85299.csv')

In [43]:
needs_metadata_check = set(needs_classifications['iauname']) - set(have_just_uploaded['iauname'])
len(needs_metadata_check)

25897

All these galaxies must have correct metadata to retire at 5 total classifications rather than 3, unless they are duplicates.

Only the enhanced workflow subjects matter as the other is a simple duplicate.

In [44]:
subjects_to_check = latest_subjects[latest_subjects['iauname'].isin(needs_metadata_check)].query('workflow_id == 10582').reset_index()
len(subjects_to_check) 
# about 1k to investigate, why did this drop?!

24986

In [45]:
assert subjects_to_check['iauname'].value_counts().max()  # no duplicates, happily

In [46]:
subjects_to_check['retirement_limit'] = subjects_to_check['metadata'].apply(lambda x: x['#retirement_limit'])
subjects_to_check['retirement_limit'].value_counts()

5    24986
Name: retirement_limit, dtype: int64

The 5k with retirement=3 need that increased to retirement=5

In [47]:
# iaunames_to_set_retirement_5 = subjects_to_check.query('retirement_limit == "3"')['iauname']
# set_retirement_5 = should_be_uploaded[should_be_uploaded['iauname'].isin(iaunames_to_set_retirement_5)]

set_retirement_5 = subjects_to_check.query('retirement_limit == "3"')

len(set_retirement_5)

0

In [48]:
subjects_to_check['subject_id'].value_counts()

47213582    1
47207878    1
47198899    1
34212532    1
34216630    1
           ..
47207760    1
47205715    1
41869379    1
42302805    1
41975808    1
Name: subject_id, Length: 24986, dtype: int64

In [49]:
# set_retirement_5.to_csv('set_retirement_5.csv', index=False)

Now let's return to the 1k drop.

These galaxies should_be_uploaded but 
- don't have >=5 classifications
- are *not* in latest_subjects


In [50]:
mysterious_1k = needs_metadata_check - set(subjects_to_check['iauname'])
len(mysterious_1k)

911


Shouldn't these have been in the batch just uploaded? Well, that only included galaxies with NO classifications. Do they have classifications? 

In [51]:
latest_classifications['iauname'].isin(mysterious_1k).sum()  # yes, they all have some classifications!

911

In [52]:
mysterious_1k_classifications = latest_classifications[latest_classifications['iauname'].isin(mysterious_1k)]
mysterious_1k_classifications['smooth-or-featured_total-votes'].value_counts()

1.0    452
2.0    293
3.0    123
4.0     43
Name: smooth-or-featured_total-votes, dtype: int64

I don't know why these guys didn't show up in the subjects export. 

Either way, they should all have retirement=5 - but I need to know which subjects these are first! 

Are they in the raw classifications export?

TODO continue

As for the rest of the galaxies - everything with >= 5 classifications and not retired needs retirement=5 and retire_now=1.

This won't include galaxies missing from the subjects export, but there's nothing I can do about that.

In [53]:
should_be_retired = latest_subjects[latest_subjects['iauname'].isin(set(enough_classifications))].query('workflow_id == "10582"')

In [54]:
assert should_be_retired['subject_id'].value_counts().max() == 1

In [55]:
should_be_retired['iauname'].value_counts().value_counts()

1    161929
2         3
Name: iauname, dtype: int64

In [59]:
pd.isna(should_be_retired['retired_at']).mean(), pd.isna(should_be_retired['retired_at']).sum()

(0.0, 0)

In [57]:
send_manual_retirement = should_be_retired[pd.isna(should_be_retired['retired_at'])]['subject_id']

In [58]:
assert False

AssertionError: 

In [ ]:
send_manual_retirement

In [ ]:
# with open('for_manual_retirement_classification_count.txt', 'w') as f:
#     for subject in send_manual_retirement:
#         f.write(str(subject) + '\n')

Only a 80% of these are actually listed as retired!

Update - now 83% are retired

In [ ]:
# should_be_retired['retirement_limit'] = should_be_retired['metadata'].apply(lambda x: x['#retirement_limit'])
should_be_retired['retire_now'] = should_be_retired['metadata'].apply(lambda x: x.get('!retire_now', 0))
# should_be_retired['retirement_limit'].value_counts()

In [ ]:
should_be_retired['retire_now'].value_counts()

About 2/3rds have retirement_limit=5, and so should retire when I re-run Caesar.

How many need updates i.e. are not retired and have retirement=3?

In [ ]:
# needs_update = (should_be_retired['retirement_limit'] == 3) & (pd.isna(should_be_retired['retired_at']))
# needs_update.mean(), needs_update.sum()  # about 30k

In [ ]:
# should_be_retired[needs_update].to_csv('set_retirement_5_retire_now_as_enough_classifications.csv', index=False)
# triggered with retire_now=1

In [ ]:
assert (pd.isna(should_be_retired['retired_at']) == pd.isna(should_be_retired['retirement_reason'])).any()  # equiv.

In [ ]:
needs_update = (should_be_retired['retire_now'] == 0) & (pd.isna(should_be_retired['retired_at']))
# pd.isna(should_be_retired['retired_at']).mean()
needs_update.mean(), needs_update.sum()  # about 30k

In [ ]:
# should_be_retired[needs_update].to_csv('set_retire_now_as_enough_classifications_not_retired.csv', index=False)

In [ ]:
expect_to_retire = (should_be_retired['retire_now'] == 1) & (pd.isna(should_be_retired['retired_at']))
expect_to_retire.sum()

In [ ]:
should_be_retired[expect_to_retire]

In [ ]:
assert False

In [60]:
latest_subjects['active'] = pd.isna(latest_subjects['retired_at'])

In [69]:
workflow = latest_subjects.query('workflow_id == 10582').reset_index()
workflow['subject_set_id'].value_counts()

74905    178513
85299     21900
Name: subject_set_id, dtype: int64

In [70]:
workflow.query('active')['subject_set_id'].value_counts()

74905    12580
85299     2550
Name: subject_set_id, dtype: int64

In [71]:
missing_subjects = set(workflow.query('active').query('subject_set_id == 85299')['iauname'])
classifications = latest_classifications[latest_classifications['iauname'].isin(missing_subjects)]
classifications['smooth-or-featured_total-votes'].value_counts()


3.0    751
4.0    702
2.0    619
1.0    374
Name: smooth-or-featured_total-votes, dtype: int64

In [72]:
workflow['retire_now'] = workflow['metadata'].apply(lambda x: x.get('!retire_now', 0))

In [74]:
(workflow['retire_now'] & workflow['active']).any()

False

In [ ]:
(should_be_retired['retire_now'] == 0).mean()

In [ ]:
not_retired = pd.isna(should_be_retired['retired_at'])
no_retirement_trigger = (should_be_retired['retire_now'] == 0)

In [ ]:
(not_retired & no_retirement_trigger).mean()

In [ ]:
(not_retired | no_retirement_trigger).mean()

In [ ]:
# latest_subjects_sat = latest_subjects
# latest_subjects_mon = latest_subjects

In [ ]:
linked_sets = [85299, 74905, 74909]

In [ ]:
latest_subjects_sat = latest_subjects_sat[latest_subjects_sat['subject_set_id'].isin(linked_sets)].reset_index()
latest_subjects_mon = latest_subjects_mon[latest_subjects_mon['subject_set_id'].isin(linked_sets)].reset_index()

In [ ]:
latest_subjects_sat['retirement_reason'].value_counts()

In [ ]:
latest_subjects_mon['retirement_reason'].value_counts()

In [ ]:
latest_subjects_sat['retirement_limit'] = latest_subjects_sat['metadata'].apply(lambda x: x['#retirement_limit'])
latest_subjects_sat['retire_now'] = latest_subjects_sat['metadata'].apply(lambda x: x.get('!retire_now', 0))
latest_subjects_sat['retirement_limit'].value_counts()

latest_subjects_mon['retirement_limit'] = latest_subjects_mon['metadata'].apply(lambda x: x['#retirement_limit'])
latest_subjects_mon['retire_now'] = latest_subjects_mon['metadata'].apply(lambda x: x.get('!retire_now', 0))
latest_subjects_mon['retirement_limit'].value_counts()

In [ ]:
latest_subjects_sat['retired'] = ~pd.isna(latest_subjects_sat['retired_at'])
latest_subjects_mon['retired'] = ~pd.isna(latest_subjects_mon['retired_at'])

In [ ]:
unretired_since_sat = latest_subjects_sat['retired'] & ~latest_subjects_mon['retired']
retired_since_sat = ~latest_subjects_sat['retired'] & latest_subjects_mon['retired']

In [ ]:
retired_since_sat.sum(), unretired_since_sat.sum()

In [ ]:
latest_subjects_sat['retirement_limit'] = latest_subjects_sat['metadata'].apply(lambda x: x['#retirement_limit'])
latest_subjects_sat['retire_now'] = latest_subjects_sat['metadata'].apply(lambda x: x.get('!retire_now', 0))
latest_subjects_sat['retirement_limit'].value_counts()

In [ ]:
latest_subjects_mon['retirement_limit'] = latest_subjects_mon['metadata'].apply(lambda x: x['#retirement_limit'])
latest_subjects_mon['retire_now'] = latest_subjects_mon['metadata'].apply(lambda x: x.get('!retire_now', 0))
latest_subjects_mon['retirement_limit'].value_counts()

In [ ]:
latest_subjects_sat['retire_now'].value_counts()

In [ ]:
latest_subjects_mon['retire_now'].value_counts()

In [ ]:
latest_subjects_mon[retired_since_sat]['retirement_limit'].value_counts()

In [ ]:
latest_subjects_mon[retired_since_sat]['retire_now'].value_counts()

In [ ]:
((latest_subjects_mon[retired_since_sat]['retirement_limit'] == 5) | (latest_subjects_mon[retired_since_sat]['retire_now'] == 1)).mean()

In [ ]:
((latest_subjects_mon[retired_since_sat]['retirement_limit'] == 3) & (latest_subjects_mon[retired_since_sat]['retirement_reason'] == 'classification_count')).mean()

In [ ]:
((latest_subjects_mon['retirement_limit'] == 3)  & (~latest_subjects_mon['retired']) & (~latest_subjects_mon['retire_now'] == 1)).sum()

In [ ]:
((latest_subjects_sat['retire_now'] == 0) & (latest_subjects_mon['retire_now'] == 1)).sum()

In [ ]:
((latest_subjects_sat['retire_now'] == 0) & (latest_subjects_mon['retire_now'] == 1) & (latest_subjects_mon['retirement_reason'] == 'flagged')).sum()

In [ ]:
latest_subjects_sat[(latest_subjects_sat['retire_now'] == 0) & (latest_subjects_mon['retire_now'] == 1) & (latest_subjects_mon['retirement_reason'] == 'flagged')]['subject_id']

In [ ]:
latest_subjects_mon[(latest_subjects_mon['retire_now'] == 1) & (~latest_subjects_mon['retired'])]['subject_id']

In [ ]:
latest_subjects_mon.query('subject_id == 32497930').iloc[0]['metadata']

In [ ]:
len(latest_subjects_sat) == len(latest_subjects_mon)

In [ ]:
latest_subjects['retirement_reason'].value_counts()

In [ ]:
json.loads(latest_subjects.query('subject_id == "47209182"')['locations'].iloc[0])['0']

In [ ]:
latest_subjects['metadata'].values[-1]

In [ ]:
latest_classifications['smooth-or-featured_total-votes'].sum()

In [ ]:
eagle = pd.read_csv('/media/walml/beta/misc_downloads/galaxy-zoo-subjects_2020_10_24.csv').query('subject_set_id == "77652"')

In [ ]:
eagle

In [ ]:
eagle['classifications_count'].hist()

In [ ]:
pd.isna(eagle['retired_at']).mean()

In [ ]:
pd.isna(latest_subjects.query('subject_set_id == "74905"')['retired_at']).sum()

In [ ]:
pd.isna(should_be_retired.query('subject_set_id == "74905"')['retired_at']).sum()

In [ ]:
all_s = pd.read_csv('/media/walml/beta/misc_downloads/galaxy-zoo-subjects_2020_10_24.csv')


In [ ]:

s = [39357184,41985466,39690308,40354405,32503186]

all_s[all_s['subject_id'].isin(s)]['subject_set_id'].value_counts()